## DVOA Scraping
The results from this are saved on S3.

In [1]:
import requests
from bs4 import BeautifulSoup
from typing import List
import pandas as pd

In [219]:
def create_historical_urls(start: int, end: int = 0) -> List:
    # This is the easiest way to do it, just handle the errors downstream.
    #    The number of weeks varies by year (like 1993 had 18 weeks with 2 byes) so this is easiest.
    weeks=18
    urls = []
    if end==0:
        end=start
    for year in range(start, end+1):
        for week in range(1, weeks+1):
            urls.append(f'https://www.footballoutsiders.com/stats/nfl/historical-lookup-by-week/{year}/{week}/overall')
            
    return urls   

In [220]:
def login_and_request(data_urls: List[str], login_url: str, payload: dict) -> pd.DataFrame:
    with requests.Session() as s:
        p = s.post(login_url, data=payload)
        df_initiated = False
        for i, url in enumerate(data_urls):
            year = url.split('/')[-3]
            week = url.split('/')[-2]
        
            r = s.get(url, cookies=p.cookies)
            soup = BeautifulSoup(r.content, 'html.parser')
            evens = soup.find_all('tr', class_='even')
            odds = soup.find_all('tr', class_='odd')
            
            # Account for invalid weeks, which will have a one-row table
            if (len(evens)==0) or (len(odds)==0):
                print(f'No data for {year} season, week {week}!')
                continue
            
            if not df_initiated:
                headers = soup.find_all('thead')
                headers_list = [item.text for item in headers[0].find_all('th')]
                headers_list.insert(0, 'Week')
                headers_list.insert(0, 'Year')
                
                rank_headers = [head + ' (Rank)' for head in headers_list[-8:]]
                headers_list.extend(rank_headers)
                
                results_df = pd.DataFrame(columns=headers_list)
                df_initiated = True
            
            for group in [evens, odds]:
                for data_row in group:
                    team = data_row.find_all('td')[0].text
                    record = data_row.find_all('td')[1].text
                    # Going to just format them as floats from the start
                    dvoa_row = [float(item.text[:-1]) for item in data_row.find_all('td', class_='new-table-right-joined')]
                    rank_row = [int(item.text) for item in data_row.find_all('td', class_='new-table-left-joined')]
                    dvoa_row.extend(rank_row)
                    # This is pretty confusing, should probably just append and then re-arrange column order at the end.
                    dvoa_row.insert(0, record)
                    dvoa_row.insert(0, team)
                    dvoa_row.insert(0, week)
                    dvoa_row.insert(0, year)
                    results_df = results_df.append(dict(zip(headers_list, dvoa_row)), ignore_index=True)
 
            print(f'Completed processing for {len(evens) + len(odds)} teams for {year} season, week {week}.')
    
    return results_df

In [221]:
login_url = 'https://www.footballoutsiders.com/user/login'
test_url = 'https://www.footballoutsiders.com/stats/nfl/historical-lookup-by-week/2022/10/overall'
payload = {'name': 'the_raw', 'pass': 'yV9K7bN^qMFP', 'form_build_id': 'form--_HbxtM_HeEnx4bhUVx4IswEFwSasETNCicodZFlhtA', 'form_id': 'user_login_form', 'op': 'Login'}

In [46]:
year = test_url.split('/')[-3]
week = test_url.split('/')[-2]

## Note on payload for login
Had to inspect the source code for the page and find the login form.  It wouldn't work until I had a key/value for every "name"/"value" within an input tag.

Note that they have DAVE for 2022, so will need to filter that out when I get data for the current season.

In [11]:
data_urls = create_historical_urls(2000, 2021)

In [55]:
dvoas_21st_century = login_and_request(data_urls, login_url, payload)

Completed processing for 31 teams for week 1.
Completed processing for 31 teams for week 2.
Completed processing for 31 teams for week 3.
Completed processing for 31 teams for week 4.
Completed processing for 31 teams for week 5.
Completed processing for 31 teams for week 6.
Completed processing for 31 teams for week 7.
Completed processing for 31 teams for week 8.
Completed processing for 31 teams for week 9.
Completed processing for 31 teams for week 10.
Completed processing for 31 teams for week 11.
Completed processing for 31 teams for week 12.
Completed processing for 31 teams for week 13.
Completed processing for 31 teams for week 14.
Completed processing for 31 teams for week 15.
Completed processing for 31 teams for week 16.
Completed processing for 31 teams for week 17.
Completed processing for 31 teams for week 18.
Completed processing for 31 teams for week 19.
Completed processing for 31 teams for week 20.
Completed processing for 31 teams for week 21.
Completed processing f

In [65]:
dvoas_21st_century.head()

,Year,Week,Team,W-L,Total DVOA,Weighted DVOA,Offense DVOA,Offense Weighted DVOA,Defense DVOA,Defense Weighted DVOA,Special Teams DVOA,Special Teams Weighted DVOA
0,2000,1,PHI,1-0,77.3,77.3,12.8,12.8,-63.5,-63.5,1.0,1.0
1,2000,1,ATL,1-0,33.9,33.9,13.0,13.0,7.4,7.4,28.3,28.3
2,2000,1,JAX,1-0,33.0,33.0,33.7,33.7,7.5,7.5,6.7,6.7
3,2000,1,IND,1-0,22.2,22.2,16.0,16.0,6.6,6.6,12.8,12.8
4,2000,1,BUF,1-0,15.4,15.4,-4.4,-4.4,-43.3,-43.3,-23.4,-23.4


In [58]:
dvoas_21st_century.describe()

,Total DVOA,Weighted DVOA,Offense DVOA,Offense Weighted DVOA,Defense DVOA,Defense Weighted DVOA,Special Teams DVOA,Special Teams Weighted DVOA
count,11966.000000,11966.000000,11966.000000,11966.000000,11966.000000,11966.000000,11966.000000,11966.000000
mean,-0.672154,-0.546289,-0.392470,-0.419104,-0.237063,-0.297560,-0.517257,-0.425280
std,24.325721,24.482180,18.148839,18.278680,14.235253,14.305721,5.769625,5.845992
min,-133.700000,-133.700000,-106.300000,-106.300000,-95.400000,-95.400000,-43.800000,-43.800000
25%,-15.700000,-15.700000,-11.500000,-11.800000,-8.600000,-8.800000,-3.600000,-3.600000
50%,-0.100000,0.200000,-0.100000,-0.200000,0.500000,0.400000,-0.500000,-0.400000
75%,15.200000,15.500000,10.900000,11.100000,8.600000,8.600000,2.700000,3.000000
max,119.200000,119.200000,100.700000,100.700000,89.200000,89.200000,51.700000,51.700000


In [63]:
dvoas_21st_century.isna().sum()

Year                           0
Week                           0
Team                           0
W-L                            0
Total DVOA                     0
Weighted DVOA                  0
Offense DVOA                   0
Offense Weighted DVOA          0
Defense DVOA                   0
Defense Weighted DVOA          0
Special Teams DVOA             0
Special Teams Weighted DVOA    0
dtype: int64

In [64]:
dvoas_21st_century.sum()

Year                           2000200020002000200020002000200020002000200020...
Week                           1111111111111111111111111111111222222222222222...
Team                           PHIATLJAXINDBUFSTLCHIDETKCDENARITENGBCARDALMIA...
W-L                            1-01-01-01-01-01-00-11-00-10-10-10-10-10-10-11...
Total DVOA                                                                 -8043
Weighted DVOA                                                            -6536.9
Offense DVOA                                                             -4696.3
Offense Weighted DVOA                                                      -5015
Defense DVOA                                                             -2836.7
Defense Weighted DVOA                                                    -3560.6
Special Teams DVOA                                                       -6189.5
Special Teams Weighted DVOA                                              -5088.9
dtype: object

In [43]:
# Think of each row as a "Performance", it's a specific week for a specific team

In [68]:
dvoas_21st_century.to_parquet('dvoa_per_performance')

In [5]:
dvoas_21st_century = pd.read_parquet('dvoa_per_performance')

In [10]:
dvoas_21st_century[dvoas_21st_century.Team=='DAL'].tail(18)

,Year,Week,Team,W-L,Total DVOA,Weighted DVOA,Offense DVOA,Offense Weighted DVOA,Defense DVOA,Defense Weighted DVOA,Special Teams DVOA,Special Teams Weighted DVOA
11400,2021,1,DAL,0-1,-8.3,-8.3,23.0,23.0,13.6,13.6,-17.7,-17.7
11428,2021,2,DAL,1-1,7.6,7.6,24.4,24.4,10.5,10.5,-6.3,-6.3
11475,2021,3,DAL,2-1,18.8,18.8,23.1,23.1,1.2,1.2,-3.1,-3.1
11486,2021,4,DAL,3-1,32.2,32.2,27.8,27.8,-6.1,-6.1,-1.7,-1.7
11518,2021,5,DAL,4-1,32.4,32.4,22.3,22.3,-10.3,-10.3,-0.3,-0.3
11568,2021,6,DAL,5-1,22.6,22.9,19.0,18.9,-5.3,-5.4,-1.7,-1.4
11600,2021,7,DAL,5-1,24.1,24.2,20.4,20.2,-5.4,-5.5,-1.7,-1.5
11632,2021,8,DAL,6-1,25.5,25.6,19.2,18.5,-8.9,-9.2,-2.6,-2.1
11664,2021,9,DAL,6-2,19.5,18.3,14.1,12.1,-7.5,-7.9,-2.2,-1.7
11695,2021,10,DAL,7-2,26.6,26.0,14.6,12.7,-13.7,-14.8,-1.6,-1.4


In [26]:
#data_urls = create_historical_urls(2022)

In [27]:
#dvoa_2022 = login_and_request(data_urls, login_url, payload)

In [22]:
dvoa_2022

,Year,Week,Team,W-L,Total DVOA,Weighted DVOA,Offense DVOA,Offense Weighted DVOA,Defense DVOA,Defense Weighted DVOA,Special Teams DVOA,Special Teams Weighted DVOA
0,2022,1,BAL,1-0,46.0,46.0,28.6,28.6,-16.9,-16.9,0.4,0.4
1,2022,1,KC,1-0,38.7,38.7,44.5,44.5,0.5,0.5,-5.3,-5.3
2,2022,1,MIN,1-0,33.1,33.1,31.9,31.9,3.0,3.0,4.2,4.2
3,2022,1,PIT,1-0,18.8,18.8,-17.0,-17.0,-31.9,-31.9,3.8,3.8
4,2022,1,ATL,0-1,14.1,14.1,20.3,20.3,15.0,15.0,8.7,8.7
...,...,...,...,...,...,...,...,...,...,...,...,...
571,2022,18,TEN,7-10,-9.2,-13.9,-5.9,-8.2,1.8,3.6,-1.5,-2.1
572,2022,18,DEN,5-12,-11.3,-13.8,-13.1,-12.7,-5.2,-0.8,-3.4,-2.0
573,2022,18,MIN,13-4,-13.6,-18.4,-3.1,-5.8,6.7,7.9,-3.8,-4.6
574,2022,18,ARI,4-13,-22.4,-20.5,-15.9,-16.2,5.5,4.6,-1.0,0.3


In [24]:
dvoa_2000_2022 = pd.concat([dvoas_21st_century, dvoa_2022])

In [28]:
import warnings
warnings.filterwarnings('ignore')

In [94]:
data_urls = create_historical_urls(1981, 1999)

In [95]:
dvoa_old = login_and_request(data_urls, login_url, payload)

Completed processing for 28 teams for 1981 season, week 1.
Completed processing for 56 teams for 1981 season, week 2.
Completed processing for 56 teams for 1981 season, week 3.
Completed processing for 56 teams for 1981 season, week 4.
Completed processing for 56 teams for 1981 season, week 5.
Completed processing for 56 teams for 1981 season, week 6.
Completed processing for 56 teams for 1981 season, week 7.
Completed processing for 56 teams for 1981 season, week 8.
Completed processing for 56 teams for 1981 season, week 9.
Completed processing for 56 teams for 1981 season, week 10.
Completed processing for 56 teams for 1981 season, week 11.
Completed processing for 56 teams for 1981 season, week 12.
Completed processing for 56 teams for 1981 season, week 13.
Completed processing for 56 teams for 1981 season, week 14.
Completed processing for 56 teams for 1981 season, week 15.
Completed processing for 56 teams for 1981 season, week 16.
No data for 1981 season, week 17!
No data for 198

In [96]:
safety_copy = dvoa_old.copy()

In [107]:
dvoa_old['Team'].value_counts()

SD\n      313
BUF       313
NYJ       313
WAS       313
SF        313
NYG       313
SEA       313
MIN       313
TB        313
ATL       313
DET       313
MIA       313
DAL       313
GB        313
KC        313
NE        313
PHI       313
PIT       313
NO        313
CHI       313
CIN       313
DEN       313
IND       266
HOIL\n    262
CLE1\n    245
LARM\n    228
LARD\n    197
OAK       116
STLC\n    111
ARI       102
PHX\n     100
STL\n      85
CAR        85
JAX        85
BAL        68
TEN        51
BALC\n     47
CLE        17
Name: Team, dtype: int64

In [118]:
dvoa_old['Team'] = dvoa_old['Team'].str.strip('\n')

In [109]:
football_outsiders_map

{'There is no data to show.': False}

In [123]:
football_outsiders_map = {team: team for team in dvoa_old['Team'].unique()}
conversions = {
    'SD': 'LAC',
    'HOIL': 'TEN',
    'CLE1': 'CLE',
    'LARM': 'LAR',
    'LARD': 'LVR',
    'STLC': 'ARI',
    'PHX': 'ARI',
    'STL': 'LAR',
    'OAK': 'LVR',
    'BALC': 'IND'
}
for old in conversions:
    football_outsiders_map[old] = conversions[old]

In [125]:
len(football_outsiders_map)

38

In [126]:
dvoa_old['Team'] = dvoa_old['Team'].map(football_outsiders_map)

In [127]:
len(dvoa_old['Team'].unique())

31

In [133]:
dvoa_old['key'] = dvoa_old['Year'].astype('str') + dvoa_old['Week'].astype('str') + dvoa_old['Team']

In [135]:
dvoa_old.shape

(8951, 13)

In [139]:
dvoa_old = dvoa_old.drop_duplicates('key')

In [141]:
dvoa_old.groupby('Week').count()

,Year,Team,W-L,Total DVOA,Weighted DVOA,Offense DVOA,Offense Weighted DVOA,Defense DVOA,Defense Weighted DVOA,Special Teams DVOA,Special Teams Weighted DVOA,key
Week,,,,,,,,,,,,
1,515,515,515,515,515,515,515,515,515,515,515,515
10,515,515,515,515,515,515,515,515,515,515,515,515
11,515,515,515,515,515,515,515,515,515,515,515,515
12,515,515,515,515,515,515,515,515,515,515,515,515
13,515,515,515,515,515,515,515,515,515,515,515,515
14,515,515,515,515,515,515,515,515,515,515,515,515
15,515,515,515,515,515,515,515,515,515,515,515,515
16,515,515,515,515,515,515,515,515,515,515,515,515
17,291,291,291,291,291,291,291,291,291,291,291,291


In [142]:
dvoa_old = dvoa_old.drop(columns='key')

In [146]:
dvoa_all = pd.concat([dvoa_2000_2022, dvoa_old])

In [147]:
dvoa_all

,Year,Week,Team,W-L,Total DVOA,Weighted DVOA,Offense DVOA,Offense Weighted DVOA,Defense DVOA,Defense Weighted DVOA,Special Teams DVOA,Special Teams Weighted DVOA
0,2000,1,PHI,1-0,77.3,77.3,12.8,12.8,-63.5,-63.5,1.0,1.0
1,2000,1,ATL,1-0,33.9,33.9,13.0,13.0,7.4,7.4,28.3,28.3
2,2000,1,JAX,1-0,33.0,33.0,33.7,33.7,7.5,7.5,6.7,6.7
3,2000,1,IND,1-0,22.2,22.2,16.0,16.0,6.6,6.6,12.8,12.8
4,2000,1,BUF,1-0,15.4,15.4,-4.4,-4.4,-43.3,-43.3,-23.4,-23.4
...,...,...,...,...,...,...,...,...,...,...,...,...
8946,1999,17,NYG,7-9,-11.0,-10.9,-6.9,-2.3,3.1,6.5,-1.1,-2.1
8947,1999,17,CHI,6-10,-13.3,-14.3,-3.8,-4.0,6.5,6.7,-3.1,-3.6
8948,1999,17,ARI,6-10,-29.5,-27.7,-23.2,-16.3,6.1,11.9,-0.2,0.5
8949,1999,17,CIN,4-12,-33.9,-25.5,-10.0,-5.7,17.5,17.0,-6.4,-2.8


In [66]:
import numpy as np

In [70]:
np.load('old_name_map.npy', allow_pickle=True)

array({'Arizona Cardinals': 'Arizona Cardinals', 'Phoenix Cardinals': 'Arizona Cardinals', 'St. Louis Cardinals': 'Arizona Cardinals', 'Atlanta Falcons': 'Atlanta Falcons', 'Baltimore Ravens': 'Baltimore Ravens', 'Buffalo Bills': 'Buffalo Bills', 'Carolina Panthers': 'Carolina Panthers', 'Chicago Bears': 'Chicago Bears', 'Cincinnati Bengals': 'Cincinnati Bengals', 'Cleveland Browns': 'Cleveland Browns', 'Dallas Cowboys': 'Dallas Cowboys', 'Denver Broncos': 'Denver Broncos', 'Detroit Lions': 'Detroit Lions', 'Green Bay Packers': 'Green Bay Packers', 'Houston Texans': 'Houston Texans', 'Baltimore Colts': 'Indianapolis Colts', 'Indianapolis Colts': 'Indianapolis Colts', 'Jacksonville Jaguars': 'Jacksonville Jaguars', 'Kansas City Chiefs': 'Kansas City Chiefs', 'Los Angeles Chargers': 'Los Angeles Chargers', 'San Diego Chargers': 'Los Angeles Chargers', 'Los Angeles Rams': 'Los Angeles Rams', 'St. Louis Rams': 'Los Angeles Rams', 'Miami Dolphins': 'Miami Dolphins', 'Minnesota Vikings': 'Mi

In [71]:
np.load('names_2020.npy', allow_pickle=True)

array({'ARI': 'Arizona Cardinals', 'ATL': 'Atlanta Falcons', 'BAL': 'Baltimore Ravens', 'BUF': 'Buffalo Bills', 'CAR': 'Carolina Panthers', 'CHI': 'Chicago Bears', 'CIN': 'Cincinnati Bengals', 'CLE': 'Cleveland Browns', 'DAL': 'Dallas Cowboys', 'DEN': 'Denver Broncos', 'DET': 'Detroit Lions', 'GB': 'Green Bay Packers', 'HOU': 'Houston Texans', 'IND': 'Indianapolis Colts', 'JAX': 'Jacksonville Jaguars', 'KC': 'Kansas City Chiefs', 'LAC': 'Los Angeles Chargers', 'LAR': 'Los Angeles Rams', 'LVR': 'Las Vegas Raiders', 'MIA': 'Miami Dolphins', 'MIN': 'Minnesota Vikings', 'NE': 'New England Patriots', 'NO': 'New Orleans Saints', 'NYG': 'New York Giants', 'NYJ': 'New York Jets', 'PHI': 'Philadelphia Eagles', 'PIT': 'Pittsburgh Steelers', 'SEA': 'Seattle Seahawks', 'SF': 'San Francisco 49ers', 'TB': 'Tampa Bay Buccaneers', 'TEN': 'Tennessee Titans', 'WAS': 'Washington Football Team'},
      dtype=object)

In [72]:
np.load('shorthand_map.npy', allow_pickle=True)

array({'Arizona Cardinals': 'ARI', 'Phoenix Cardinals': 'ARI', 'St. Louis Cardinals': 'ARI', 'Atlanta Falcons': 'ATL', 'Baltimore Ravens': 'BAL', 'Buffalo Bills': 'BUF', 'Carolina Panthers': 'CAR', 'Chicago Bears': 'CHI', 'Cincinnati Bengals': 'CIN', 'Cleveland Browns': 'CLE', 'Dallas Cowboys': 'DAL', 'Denver Broncos': 'DEN', 'Detroit Lions': 'DET', 'Green Bay Packers': 'GB', 'Houston Texans': 'HOU', 'Baltimore Colts': 'IND', 'Indianapolis Colts': 'IND', 'Jacksonville Jaguars': 'JAX', 'Kansas City Chiefs': 'KC', 'Los Angeles Chargers': 'LAC', 'San Diego Chargers': 'LAC', 'Los Angeles Rams': 'LAR', 'St. Louis Rams': 'LAR', 'Miami Dolphins': 'MIA', 'Minnesota Vikings': 'MIN', 'New England Patriots': 'NE', 'Boston Patriots': 'NE', 'New Orleans Saints': 'NO', 'New York Giants': 'NYG', 'New York Jets': 'NYJ', 'Los Angeles Raiders': 'LVR', 'Las Vegas Raiders': 'LVR', 'Oakland Raiders': 'LVR', 'Philadelphia Eagles': 'PHI', 'Pittsburgh Steelers': 'PIT', 'Seattle Seahawks': 'SEA', 'San Francisc

In [73]:
np.load('shorthand_names.npy', allow_pickle=True)

array({'49ers': 'San Francisco 49ers', 'Bears': 'Chicago Bears', 'Bengals': 'Cincinnati Bengals', 'Bills': 'Buffalo Bills', 'Broncos': 'Denver Broncos', 'Browns': 'Cleveland Browns', 'Buccaneers': 'Tampa Bay Buccaneers', 'Cardinals': 'Arizona Cardinals', 'Chargers': 'Los Angeles Chargers', 'Chiefs': 'Kansas City Chiefs', 'Colts': 'Indianapolis Colts', 'Cowboys': 'Dallas Cowboys', 'Dolphins': 'Miami Dolphins', 'Eagles': 'Philadelphia Eagles', 'Falcons': 'Atlanta Falcons', 'Giants': 'New York Giants', 'Jaguars': 'Jacksonville Jaguars', 'Jets': 'New York Jets', 'Lions': 'Detroit Lions', 'Oilers': 'Tennessee Titans', 'Packers': 'Green Bay Packers', 'Panthers': 'Carolina Panthers', 'Patriots': 'New England Patriots', 'Raiders': 'Las Vegas Raiders', 'Rams': 'Los Angeles Rams', 'Ravens': 'Baltimore Ravens', 'Saints': 'New Orleans Saints', 'Seahawks': 'Seattle Seahawks', 'Steelers': 'Pittsburgh Steelers', 'Texans': 'Houston Texans', 'Titans': 'Tennessee Titans', 'Vikings': 'Minnesota Vikings',

## Fresh as of August 2023
I had been doing it in chunks (modern, old, etc.) but I'm just going to get it all at once now and it should work out fine.

In [222]:
data_urls = data_urls = create_historical_urls(1981, 2022)
all_dvoas_2022 = login_and_request(data_urls, login_url, payload)

Completed processing for 28 teams for 1981 season, week 1.
Completed processing for 56 teams for 1981 season, week 2.
Completed processing for 56 teams for 1981 season, week 3.
Completed processing for 56 teams for 1981 season, week 4.
Completed processing for 56 teams for 1981 season, week 5.
Completed processing for 56 teams for 1981 season, week 6.
Completed processing for 56 teams for 1981 season, week 7.
Completed processing for 56 teams for 1981 season, week 8.
Completed processing for 56 teams for 1981 season, week 9.
Completed processing for 56 teams for 1981 season, week 10.
Completed processing for 56 teams for 1981 season, week 11.
Completed processing for 56 teams for 1981 season, week 12.
Completed processing for 56 teams for 1981 season, week 13.
Completed processing for 56 teams for 1981 season, week 14.
Completed processing for 56 teams for 1981 season, week 15.
Completed processing for 56 teams for 1981 season, week 16.
No data for 1981 season, week 17!
No data for 198

In [223]:
safety_copy = all_dvoas_2022.copy()

In [234]:
#all_dvoas_2022 = safety_copy.copy()

In [235]:
all_dvoas_2022['Team'] = all_dvoas_2022['Team'].str.strip('\n')

In [236]:
football_outsiders_map = {team: team for team in all_dvoas_2022['Team'].unique()}
conversions = {
    'SD': 'LAC',
    'HOIL': 'TEN',
    'CLE1': 'CLE',
    'LARM': 'LAR',
    'LARD': 'LV',
    'STLC': 'ARI',
    'PHX': 'ARI',
    'STL': 'LAR',
    'OAK': 'LV',
    'BALC': 'IND',
}
for old in conversions:
    football_outsiders_map[old] = conversions[old]

In [237]:
all_dvoas_2022['Team'] = all_dvoas_2022['Team'].map(football_outsiders_map)

In [238]:
all_dvoas_2022['key'] = all_dvoas_2022['Year'].astype('str') + all_dvoas_2022['Week'].astype('str') + all_dvoas_2022['Team']

In [239]:
all_dvoas_2022.shape

(21521, 21)

In [240]:
all_dvoas_2022 = all_dvoas_2022.drop_duplicates('key')

In [241]:
all_dvoas_2022.shape

(21101, 21)

In [242]:
all_dvoas_2022.groupby('Week').count()

,Year,Team,W-L,Total DVOA,Weighted DVOA,Offense DVOA,Offense Weighted DVOA,Defense DVOA,Defense Weighted DVOA,Special Teams DVOA,Special Teams Weighted DVOA,Total DVOA (Rank),Weighted DVOA (Rank),Offense DVOA (Rank),Offense Weighted DVOA (Rank),Defense DVOA (Rank),Defense Weighted DVOA (Rank),Special Teams DVOA (Rank),Special Teams Weighted DVOA (Rank),key
Week,,,,,,,,,,,,,,,,,,,,
1,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249
10,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249
11,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249
12,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249
13,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249
14,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249
15,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249
16,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249,1249
17,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025


In [243]:
all_dvoas_2022['Team'].value_counts()

LAC    692
DAL    692
NYJ    692
WAS    692
SF     692
NYG    692
SEA    692
MIN    692
TB     692
IND    692
GB     692
DET    692
MIA    692
ATL    692
TEN    692
BUF    692
ARI    692
PHI    692
KC     692
CIN    692
NE     692
DEN    692
CHI    692
PIT    692
LAR    692
LV     692
NO     692
CLE    641
CAR    478
JAX    478
BAL    461
HOU    359
Name: Team, dtype: int64

In [244]:
len(all_dvoas_2022['Team'].unique())

32

In [245]:
#all_dvoas_2022.to_parquet('all_dvoas_through_2022.parquet', index=False)